In [17]:
df = pd.read_csv('C:/Users/boual/Documents/5DS1/ProjectAI/final_data.csv')
df

,Concepts,Text,Refrences,def,Keywords,Synonyms
0,plan risk management,plan risk management is the process of deﬁning...,captures/page_8_image_1.png,The purpose of this risk management process is...,"['project risk management process', 'plan comp...",['The PLAN RISK MANAGEMENT is a critical compo...
1,project charter,described in section 4.1.3.1. the project char...,sections/4.1.3.1_page117.png,This section describes the project charter. It...,['project charter documents highlevel'],['The PROJECT CHARTER is the most recent proje...
2,project management plan,described in section 4.2.3.1. in planning proj...,sections/4.2.3.1_page122.png,"In planning project risk management, the risk ...","['planning project risk management', 'approved...",['The PROJECT MANAGEMENT PLAN is designed to m...
3,project documents,project documents that can be considered as in...,sections/13.1.3.1_page550.png,This chapter describes the stakeholder registr...,"['projects stakeholders provides overview', 'd...",['The project documents are part of the Projec...
4,enterprise environmental factors,the enterprise environmental factors that can ...,,"The enterprise ""electrified factors"" that can ...","['plan risk management process', 'factors inﬂu...",['Enterprising enterprise environmental factor...
...,...,...,...,...,...,...
84,work performance information,described in section 4.5.1.3. work performance...,sections/4.5.1.3_page145.png,This section describes the process for identif...,"['project risk management performing', 'perfor...","['Work performance performance information.', ..."
85,change requests,described in section 4.3.3.4. the monitor risk...,sections/4.3.3.4_page132.png,A change request from a project manager must b...,"['monitor risks process', 'section 46 change',...","['Change requests are subject to change.', 'Ch..."
86,project management plan updates,any change to the project management plan goes...,,A change to the management plan is made throug...,"['change project management plan', 'request af...",['A PROJECT MANAGEMENT PLAN UPDATES FOR MODEL ...
87,project documents updates,project documents that may be updated as a res...,"sections/4.1.3.2_page117.png, sections/4.3.3....",The risk report is an updated version of the p...,"['projects risk register described', 'generate...",['Project projects are subject to project docu...


# Rebel

***Model and Tokenizer Loading:***
The code loads a pre-trained Seq2Seq model ("rebel-large") and its corresponding tokenizer from the Hugging Face Model Hub.

In [18]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


***Relation Extraction Function:***
This function takes the model's output text and extracts relations from it. It looks for specific tokens such as "<triplet>", "<subj>", and "<obj>" in the text and extracts subject, relation, and object information.

In [19]:
def extract_relations_from_model_output(text):
    relations = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")
    for token in text_replaced.split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        relations.append({
            'head': subject.strip(),
            'type': relation.strip(),
            'tail': object_.strip()
        })
    return relations

***Knowledge Base (KB) Class:****
This class represents a knowledge base, maintaining a list of relations. It has methods to check if a relation already exists, add a relation, and save the relations.


In [20]:
class KB():
    def __init__(self):
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def add_relation(self, r):
        if not self.exists_relation(r):
            self.relations.append(r)

    def save(self):
        dict_list = []
        for r in self.relations:
            dict_list.append(r)
        return(dict_list)
        #df = pd.DataFrame(dict_list, columns=['head', 'type', 'tail'])
        #print(dict_list)

***Conversion Function:***
This function takes a small text, tokenizes it, generates predictions using the loaded Seq2Seq model, extracts relations using the extract_relations_from_model_output function, and creates a knowledge base (kb) containing the extracted relations.

In [21]:
def from_small_text_to_kb(text, verbose=False):
    kb = KB()

    # Tokenizer text
    model_inputs = tokenizer(text, max_length=512, padding=True, truncation=True,
                            return_tensors='pt')
    if verbose:
        print(f"Num tokens: {len(model_inputs['input_ids'][0])}")

    # Generate
    gen_kwargs = {
        "max_length": 1000,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": 3
    }

    generated_tokens = model.generate(
        **model_inputs,
        **gen_kwargs,
    )

    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

    # create kb
    for sentence_pred in decoded_preds:
        relations = extract_relations_from_model_output(sentence_pred)
        for r in relations:
            kb.add_relation(r)

    return kb

***Relation Extraction from DataFrame:***
This loop processes each item in the 'Content' column of a DataFrame (df), generating relations and appending them to the relation_list.

In [23]:
from tqdm import tqdm
relation_list =[]
for item in tqdm(df['Text']):
    kb = from_small_text_to_kb(item, verbose=False) #verbose = True to return the number of tokens in each sentence
    relation_list.append(kb.save())

100%|██████████████████████████████████████████████████████████████████████████████████| 89/89 [06:06<00:00,  4.12s/it]


***Dataframe Creation:***
The code converts the relation_list into a flat list (dict_list) and creates a DataFrame (triplet) from this list with columns 'head', 'type', and 'tail'.

In [24]:
dict_list =[]
for item in tqdm(relation_list):
    for dic in item:
        dict_list.append(dic)
triplet = pd.DataFrame(dict_list, columns=['head', 'type', 'tail'])

100%|██████████████████████████████████████████████████████████████████████████████████████████| 89/89 [00:00<?, ?it/s]


In [25]:
triplet

,head,type,tail
0,plan risk management,subclass of,risk management
1,project,has part,project management plan
2,project description,part of,project charter
3,project charter,part of,project description
4,project charter,facet of,project description and boundaries
...,...,...,...
268,risk register,main subject,project risk
269,risk register,facet of,monitor risks process
270,risk register,part of,risk management plan
271,risk register,use,risk management


In [26]:
unique_triplets = triplet.drop_duplicates()

In [27]:
unique_triplets.shape

(226, 3)

In [28]:
unique_triplets

,head,type,tail
0,plan risk management,subclass of,risk management
1,project,has part,project management plan
2,project description,part of,project charter
3,project charter,part of,project description
4,project charter,facet of,project description and boundaries
...,...,...,...
264,change request,facet of,change control
265,change control,facet of,project management plan
266,change control process,facet of,project management plan
268,risk register,main subject,project risk


In [30]:
unique_heads = unique_triplets['head'].unique()
print(unique_heads)

['plan risk management' 'project' 'project description' 'project charter'
 'project management plan' 'risk thresholds' 'risk threshold'
 'risk management' 'risk register' 'enterprise risk management'
 'stakeholder analysis' 'risk management plan' 'project manager'
 'project management' 'risk breakdown structure' 'risk'
 'project requirements' 'costs' 'award' 'milestone date' 'project risk'
 'external procurement' 'procurement' 'risk database' 'benchmarking'
 'risk statement' 'checklists' 'project process controls' 'brainstorming'
 'risk checklist' 'uncertainty' 'facilitation' 'Interpersonal'
 'risk workshop' 'project team' 'project sponsor' 'cause(s)' 'cause'
 'effect(s)' 'qualitative risk analysis' 'issue log' 'issues' 'bias'
 'qualitative' 'risk analysis' 'expertise' 'expert judgment'
 'semi-structured interview' 'structured' 'timeliness' 'interpersonal'
 'qualitative analysis' 'sources of risk' 'hierarchy' 'priority level'
 'probability and impact matrix' 'cost' 'cost estimates' 'qu

In [31]:
df.fillna('', inplace=True)

df

,Concepts,Text,Refrences,def,Keywords,Synonyms
0,plan risk management,plan risk management is the process of deﬁning...,captures/page_8_image_1.png,The purpose of this risk management process is...,"['project risk management process', 'plan comp...",['The PLAN RISK MANAGEMENT is a critical compo...
1,project charter,described in section 4.1.3.1. the project char...,sections/4.1.3.1_page117.png,This section describes the project charter. It...,['project charter documents highlevel'],['The PROJECT CHARTER is the most recent proje...
2,project management plan,described in section 4.2.3.1. in planning proj...,sections/4.2.3.1_page122.png,"In planning project risk management, the risk ...","['planning project risk management', 'approved...",['The PROJECT MANAGEMENT PLAN is designed to m...
3,project documents,project documents that can be considered as in...,sections/13.1.3.1_page550.png,This chapter describes the stakeholder registr...,"['projects stakeholders provides overview', 'd...",['The project documents are part of the Projec...
4,enterprise environmental factors,the enterprise environmental factors that can ...,,"The enterprise ""electrified factors"" that can ...","['plan risk management process', 'factors inﬂu...",['Enterprising enterprise environmental factor...
...,...,...,...,...,...,...
84,work performance information,described in section 4.5.1.3. work performance...,sections/4.5.1.3_page145.png,This section describes the process for identif...,"['project risk management performing', 'perfor...","['Work performance performance information.', ..."
85,change requests,described in section 4.3.3.4. the monitor risk...,sections/4.3.3.4_page132.png,A change request from a project manager must b...,"['monitor risks process', 'section 46 change',...","['Change requests are subject to change.', 'Ch..."
86,project management plan updates,any change to the project management plan goes...,,A change to the management plan is made throug...,"['change project management plan', 'request af...",['A PROJECT MANAGEMENT PLAN UPDATES FOR MODEL ...
87,project documents updates,project documents that may be updated as a res...,"sections/4.1.3.2_page117.png, sections/4.3.3....",The risk report is an updated version of the p...,"['projects risk register described', 'generate...",['Project projects are subject to project docu...


In [32]:
# Assuming df is your initial DataFrame
# Assuming final_triplet_unique has 'head', 'type', 'tail' columns

# Add 'type' and 'tail' columns to df based on row indices
df['type'] = unique_triplets['type']
df['tail'] = unique_triplets['tail']
# Rename 'type' to 'predicate' and 'tail' to 'object'
df.rename(columns={'type': 'predicate', 'tail': 'object'}, inplace=True)

df

,Concepts,Text,Refrences,def,Keywords,Synonyms,predicate,object
0,plan risk management,plan risk management is the process of deﬁning...,captures/page_8_image_1.png,The purpose of this risk management process is...,"['project risk management process', 'plan comp...",['The PLAN RISK MANAGEMENT is a critical compo...,subclass of,risk management
1,project charter,described in section 4.1.3.1. the project char...,sections/4.1.3.1_page117.png,This section describes the project charter. It...,['project charter documents highlevel'],['The PROJECT CHARTER is the most recent proje...,has part,project management plan
2,project management plan,described in section 4.2.3.1. in planning proj...,sections/4.2.3.1_page122.png,"In planning project risk management, the risk ...","['planning project risk management', 'approved...",['The PROJECT MANAGEMENT PLAN is designed to m...,part of,project charter
3,project documents,project documents that can be considered as in...,sections/13.1.3.1_page550.png,This chapter describes the stakeholder registr...,"['projects stakeholders provides overview', 'd...",['The project documents are part of the Projec...,part of,project description
4,enterprise environmental factors,the enterprise environmental factors that can ...,,"The enterprise ""electrified factors"" that can ...","['plan risk management process', 'factors inﬂu...",['Enterprising enterprise environmental factor...,facet of,project description and boundaries
...,...,...,...,...,...,...,...,...
84,work performance information,described in section 4.5.1.3. work performance...,sections/4.5.1.3_page145.png,This section describes the process for identif...,"['project risk management performing', 'perfor...","['Work performance performance information.', ...",facet of,risk management
85,change requests,described in section 4.3.3.4. the monitor risk...,sections/4.3.3.4_page132.png,A change request from a project manager must b...,"['monitor risks process', 'section 46 change',...","['Change requests are subject to change.', 'Ch...",part of,project management plan
86,project management plan updates,any change to the project management plan goes...,,A change to the management plan is made throug...,"['change project management plan', 'request af...",['A PROJECT MANAGEMENT PLAN UPDATES FOR MODEL ...,has part,plan risk management
87,project documents updates,project documents that may be updated as a res...,"sections/4.1.3.2_page117.png, sections/4.3.3....",The risk report is an updated version of the p...,"['projects risk register described', 'generate...",['Project projects are subject to project docu...,part of,project management plan


In [34]:
# Save the updated DataFrame to a new CSV file
df.to_csv('C:/Users/boual/Documents/5DS1/ProjectAI/final_data_with_predicate.csv', index=False)